**Donatus Leo**
**23.55.1432**

In [ ]:
# import libraries
import hashlib
import random # untuk menghasilkan angka acak
import string
import json # Mengimport modul json untuk manipulasi data JSON
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

In [ ]:
# Install paket pycryptodome
!pip3 install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00


In [ ]:
import binascii
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [ ]:
# class client
class Client:
    def __init__(self):
        random=Crypto.Random.new().read
        self._private_key=RSA.generate(1024,random)
        self._public_key=self._private_key.publickey()
        self._signer=PKCS1_v1_5.new(self._private_key)

# The generated public key will used as the client's identity
# it returns HEX representation of the public key

    @property
    def identity(self): # Implementasi fungsi identity
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [ ]:
Anton = Client() # Membuat objek klien dengan nama Anton
print (Anton.identity) # Mencetak identitas dari objek Anton

30819f300d06092a864886f70d010101050003818d0030818902818100d1188d6c8a4a3e59e198b1667db5a3d9db9d04df621e7fe11bef33f2f4909b44bf75017ef252a9d20229755c918bc9bc72cf97990b7b9befe1a0a6f4be03bb6060c4fd56dec8f92f0cfce59a281a56ad20456f8251820fdf7f33165bd75bce31aec943ea7e22f33fbaf7a40462f1293e72c5bc40b6c38e6ec2f3fb89fd6499bb0203010001


In [ ]:
# Kelas Transaksi, klien akan mengirim uang ke seseorang
transactions = []
last_block_hash=""

class Transaction:

    def __init__(self, sender,recipient, value):
        # senders public key
        self.sender=sender

        #Recivers public key
        self.recipient=recipient

        # Amount to be sent
        self.value=value

        # Time of Transaction
        self.time=datetime.datetime.now()

 # Kombinasi keempat variabel ke objek
    def to_dict(self):
        if self.sender=="Genesis":
            identity="Genesis"

        else:
            identity=self.sender.identity

       # Construct the dictionary
        return collections.OrderedDict({'sender': identity,'recipient': self.recipient,'value': self.value,'time' : self.time})

      # We sign the dictionary object using the private key of the sender
    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))

        return binascii.hexlify(signer.sign(h)).decode('ascii')


def display_transaction(transaction):
         #for transaction in transactions:
        dict = transaction.to_dict()
        print ("sender: " + dict['sender'])
        print ('-----')
        print ("recipient: " + dict['recipient'])
        print ('-----')
        print ("value: " + str(dict['value']))
        print ('-----')
        print ("time: " + str(dict['time']))
        print ('-----')

In [ ]:
# Test Transaction class
Gerry = Client()
dini = Client()

# Gerry send 9 TPC coins to  dini
t = Transaction(Gerry,dini.identity,9.0)

signature = t.sign_transaction()
print (signature)

918b173abced7994958ad5538081e9f51030b8516a66f79c3ae47df494b412c103f06409ddadd10c42dfc228a158fb2c1c6f171eb6162a420b5589937dd835239f20463a0b86c264b73feba02ef58823ea3801e2dd707a2a7cabdc66c3d59e2cabad1b227938789d8303fed42921e6a086f015710522c530cbeadc1ec73c3344


In [ ]:
Robert = Client()
gerry = Client()
Dini = Client()
Budi = Client()

t1 = Transaction(Robert,Dini.identity,17.0)
t1.sign_transaction()
transactions.append(t1)

t2 = Transaction(Dini,Budi.identity,8.0)
t2.sign_transaction()
transactions.append(t2)

t3 = Transaction(gerry,Dini.identity,3.0)
t3.sign_transaction()
transactions.append(t3)

t4 = Transaction(gerry,Budi.identity,5.0)
t4.sign_transaction()
transactions.append(t4)

t5 = Transaction(Robert,Budi.identity,9.0)
t5.sign_transaction()
transactions.append(t5)

In [ ]:
for transaction in transactions:
    display_transaction(transaction)
    print ('--------------')

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100c03085d806303a673f5af2706475c7b390dd44398d6dd5199de1b7c745f4fcbb95d5856f2d03b54fca236c89af442fc9ee9727c254bca07b911457cf1332928658fd8b1d7f0d767e444150d43306097cad16e7efb52c4992c28dd9dce7d23a5e2d243c52998ad9c09d588e40e1f65b32ce3a6c6b40324e50ee2a6452cbd917e10203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181009e3af488a71b3e967f58754c3dcf2b7e36acc45f6332d6504b7ef48d30b6cea122a4ef4bbfdbbc58e48eeba63b5a418e0bf739eb2c40008249fa07ad08e3d8e67fac63f9c28964be4aac299fac705edf248196d123649f96d2fbc72f5ac3d9860a400f766b8adc60876d5ea308751888459ebfab44f9eed1be7cdfeb995f887b0203010001
-----
value: 17.0
-----
time: 2023-11-24 10:51:30.551283
-----
--------------
sender: 30819f300d06092a864886f70d010101050003818d00308189028181009e3af488a71b3e967f58754c3dcf2b7e36acc45f6332d6504b7ef48d30b6cea122a4ef4bbfdbbc58e48eeba63b5a418e0bf739eb2c40008249fa07ad08e3d8e67fac63f9c28964be4aac299fac705edf248196d123649f96d2fbc

In [ ]:
class Block:

    def __init__(self):
        self.verified_transaction=[]
        self.previous_block_hash=""
        self.Nonce=""

In [ ]:
# Create a Client object
Mahee=Client()

# Create a transaction Object
t0 = Transaction ("Genesis",Mahee.identity,500.0)

# Create a block Object
# Here we create a Genises Block
block0=Block()
block0.verified_transaction.append(t0)
block0.previous_block_hash=None
Nonce=None

# Hash the entire block
digest=hash(block0)

# Assign the hashed value to variable last_block_hash
last_block_hash=digest

In [ ]:
# TPCoins store the Blocks
TPCoins=[]

def dump_blockchain (self):
    print ("Number of blocks in the chain: " + str(len (self)))

    for x in range (len(TPCoins)): # loop through the TPCoins array

        # assign each block in TPCoins to a temp variable block_temp
        block_temp = TPCoins[x]
        print ("block # " + str(x))

        # display the transactions inside the block
        for transaction in block_temp.verified_transaction:
            display_transaction (transaction)
            print ('--------------')

        print ('=====================================')

In [ ]:
# Append the Genesis block to TPCoins
TPCoins.append(block0)
dump_blockchain(TPCoins)

Number of blocks in the chain: 1
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b4659fc384df927dfd3be2e5dea30a8351ae1883c41d2c317a7c4c8e6a44683ddb86b41b57eb5408ed3543af716c483c63cc114f04474da0f3d5752abf02af3dc2d80820988e72c972267c2f457c6b23eb2e7dd04e4553a924827ef07f0623408450f6c2197a638760901a3efce8cfa27ae196e8bb6dbbaebbe7a8226fc1a8f70203010001
-----
value: 500.0
-----
time: 2023-11-24 10:51:40.651405
-----
--------------


In [ ]:
#Mining process
#The sha256 function takes a message as a parameter, encodes it to ASCII, generates a hexadecimal digest and returns
#the value to the caller.

def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

In [ ]:
# Develop a Mine function
# Here it tries to find the solution to that one NOnce that will satisfy the hash for the block
# We send the block and the difficulty level as the parameters to the mine function.

def mine(message,difficulty=1):
    #The difficulty level needs to be greater or equal to 1, we ensure this with the following assert statement:
    assert difficulty >= 1

    #We create a prefix variable using the set difficulty level.
    prefix = '1' * difficulty

    for i in range(1000):
        digest = sha256(str(hash(message)) + str(i))

        if digest.startswith(prefix):
            print ("after " + str(i) + " iterations found nonce: "+ digest)
            return digest

In [ ]:
# Test the mine function
mine("test message", 2)

after 130 iterations found nonce: 115d2a59ad3258a401f1c3bc2d3add9788e988e734692cb63caef36a48350415


'115d2a59ad3258a401f1c3bc2d3add9788e988e734692cb63caef36a48350415'

In [ ]:
#Miner 2 adds a block
last_transaction_index=0
block = Block()

# append 3 transactions to block
for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    # validate transaction
    # if valid
    block.verified_transaction.append (temp_transaction)
    last_transaction_index += 1

# Assign the value of last_block_hash to the previous_block_hash attribute in the block
block.previous_block_hash = last_block_hash

# Send the block which has (Transactions + previous_bloc_hash) for the mine function
block.Nonce = mine (block, 2)

TPCoins.append (block)

# Hased the currrent block and store its hashed value in the last_block_hash variable
digest = hash (block)
#print("Block hash of the current Block "+ str(digest))
last_block_hash = digest

after 73 iterations found nonce: 114004aeca46bebc9dddf6ff37fd84990e540290686075f5848197d374f37e75


In [ ]:
dump_blockchain(TPCoins)

Number of blocks in the chain: 2
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b4659fc384df927dfd3be2e5dea30a8351ae1883c41d2c317a7c4c8e6a44683ddb86b41b57eb5408ed3543af716c483c63cc114f04474da0f3d5752abf02af3dc2d80820988e72c972267c2f457c6b23eb2e7dd04e4553a924827ef07f0623408450f6c2197a638760901a3efce8cfa27ae196e8bb6dbbaebbe7a8226fc1a8f70203010001
-----
value: 500.0
-----
time: 2023-11-24 10:51:40.651405
-----
--------------
block # 1
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100c03085d806303a673f5af2706475c7b390dd44398d6dd5199de1b7c745f4fcbb95d5856f2d03b54fca236c89af442fc9ee9727c254bca07b911457cf1332928658fd8b1d7f0d767e444150d43306097cad16e7efb52c4992c28dd9dce7d23a5e2d243c52998ad9c09d588e40e1f65b32ce3a6c6b40324e50ee2a6452cbd917e10203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181009e3af488a71b3e967f58754c3dcf2b7e36acc45f6332d6504b7ef48d30b6cea122a4ef4bbfdbbc58e48eeba63b5a418e0bf739